In [21]:
# Natural Language Processing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import json
import string
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import r2_score

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
jokes_df = pd.read_csv("jokes.csv")

### mapping jokes from jokes.csv

In [4]:
train_df.head()

,id,user_id,joke_id,Rating
0,31030_110,31030,110,2.750
1,16144_109,16144,109,5.094
2,23098_6,23098,6,-6.438
3,14273_86,14273,86,4.406
4,18419_134,18419,134,9.375


In [5]:
test_df.head()

,id,user_id,joke_id
0,6194_11,6194,11
1,19356_3,19356,3
2,23426_79,23426,79
3,40030_3,40030,3
4,19806_115,19806,115


In [6]:
jokes_df.head()

,joke_id,joke_text
0,1,Q. What's O. J. Simpson's web address? A. Slas...
1,2,How many feminists does it take to screw in a ...
2,3,Q. Did you hear about the dyslexic devil worsh...
3,4,They asked the Japanese visitor if they have e...
4,5,Q: What did the blind person say when given so...


In [7]:
train_df['joke_id'] = train_df['joke_id'].map(jokes_df.set_index('joke_id')['joke_text'])
test_df['joke_id'] = test_df['joke_id'].map(jokes_df.set_index('joke_id')['joke_text'])
sample_submission = pd.read_csv("sample_submission.csv")

In [8]:
#setup values for vectorization
## Parameters 
embed_size = 300 # how big is each word vector
max_features = 60000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 26 # max number of words in a question to use
batch_size = 3636

In [9]:
jokes_parsing = pd.DataFrame()
jokes_parsing['joke_id']=train_df['joke_id']
jokes_parsing['word_count'] = train_df['joke_id'].apply(lambda x: len(str(x).split(" ")))
jokes_parsing[['joke_id','word_count']].head()

,joke_id,word_count
0,"Judy was having trouble with her computer, so ...",109
1,One day the first grade teacher was reading th...,105
2,Q. What is orange and sounds like a parrot? A....,12
3,Two attorneys went into a diner and ordered tw...,53
4,A man is driving in the country one evening wh...,157


In [10]:
#preprocessing text 

In [11]:
temp_joke = train_df["joke_id"]
temp = []

In [12]:
import re
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

remove_special_characters("Well this was fun! What do you think? "" 123#@!", 
                          remove_digits=True)

'Well this was fun What do you think  '

In [13]:
lenth = train_df.size
leno = 0
for i in train_df["joke_id"]:
    if lenth!=leno:
        temp.append(remove_special_characters(i, remove_digits=True))

In [14]:
train_df["joke_id"] = temp

In [15]:
temp = []
lenth = test_df.size
leno = 0
for i in test_df["joke_id"]:
    if lenth!=leno:
        temp.append(remove_special_characters(i, remove_digits=True))

In [16]:
test_df["joke_id"] = temp

In [17]:
## fill up the missing values
train_X = train_df["joke_id"].fillna("_na_").values
test_X = test_df["joke_id"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)

In [22]:
## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [23]:
train_X.shape,test_X.shape

((1092059, 26), (537880, 26))

In [24]:
train_y = train_df['Rating'].values

In [25]:
# Fitting Decision Tree Regression to the dataset
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(train_X, train_y)
y_pred = regressor.predict(train_X)
Dtree_score = r2_score(train_y, y_pred)
print("DecisionTree score", Dtree_score)

DecisionTree score 0.09573864784423547


In [26]:
y_pred = regressor.predict(test_X)

In [27]:
sample_submission.head()

,id,Rating
0,6194_11,3.024129
1,19356_3,-0.671942
2,23426_79,3.637524
3,40030_3,-0.671942
4,19806_115,2.841260


In [28]:
sample_submission["Rating"]=y_pred

In [29]:
sample_submission.to_csv("sample_submission.csv")